In [32]:
import time
import argparse
import numpy as np
import torch
import torch.nn.functional as F
import torch.optim as optim
import pickle as pkl
from time import perf_counter
import torch.nn as nn
from torch.nn import Module
import torch.nn.functional as F
import math
import sys
import scipy.sparse as sp
import networkx as nx
import matplotlib.pyplot as plt

In [31]:
from sklearn.metrics import f1_score

def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)

def f1(output, labels):
    preds = output.max(1)[1]
    preds = preds.cpu().detach().numpy()
    labels = labels.cpu().detach().numpy()
    micro = f1_score(labels, preds, average='micro')
    macro = f1_score(labels, preds, average='macro')
    return micro, macro


In [8]:
class SGC(nn.Module):
    """
    A Simple PyTorch Implementation of Logistic Regression.
    Assuming the features have been preprocessed with k-step graph propagation.
    """
    def __init__(self, nfeat, nclass):
        super(SGC, self).__init__()

        self.W = nn.Linear(nfeat, nclass)

    def forward(self, x):
        return self.W(x)

In [33]:
def parse_index_file(filename):
    """Parse index file."""
    index = []
    for line in open(filename):
        index.append(int(line.strip()))
    return index

def row_normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

def normalization(adj):
   adj = adj + sp.eye(adj.shape[0])
   adj = sp.coo_matrix(adj)
   row_sum = np.array(adj.sum(1))
   d_inv_sqrt = np.power(row_sum, -0.5).flatten()
   d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
   d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
   return d_mat_inv_sqrt.dot(adj).dot(d_mat_inv_sqrt).tocoo()

def preprocess_citation(adj, features):
    adj = normalization(adj)
    features = row_normalize(features)
    return adj, features

def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

def load_citation(dataset_str="cora", cuda=True):
    """
    Load Citation Networks Datasets.
    """
    names = ['x', 'y', 'tx', 'ty', 'allx', 'ally', 'graph']
    objects = []
    for i in range(len(names)):
        with open("data/ind.{}.{}".format(dataset_str.lower(), names[i]), 'rb') as f:
            if sys.version_info > (3, 0):
                objects.append(pkl.load(f, encoding='latin1'))
            else:
                objects.append(pkl.load(f))

    x, y, tx, ty, allx, ally, graph = tuple(objects)
    test_idx_reorder = parse_index_file("data/ind.{}.test.index".format(dataset_str))
    test_idx_range = np.sort(test_idx_reorder)

    features = sp.vstack((allx, tx)).tolil()
    features[test_idx_reorder, :] = features[test_idx_range, :]
    adj = nx.adjacency_matrix(nx.from_dict_of_lists(graph))
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)
    labels = np.vstack((ally, ty))
    labels[test_idx_reorder, :] = labels[test_idx_range, :]

    idx_test = test_idx_range.tolist()
    idx_train = range(len(y))
    idx_val = range(len(y), len(y)+500)

    adj, features = preprocess_citation(adj, features)

    # porting to pytorch
    features = torch.FloatTensor(np.array(features.todense())).float()
    labels = torch.LongTensor(labels)
    labels = torch.max(labels, dim=1)[1]
    adj = sparse_mx_to_torch_sparse_tensor(adj).float()
    idx_train = torch.LongTensor(idx_train)
    idx_val = torch.LongTensor(idx_val)
    idx_test = torch.LongTensor(idx_test)

    return adj, features, labels, idx_train, idx_val, idx_test

def sgc_precompute(features, adj, degree):
    t = perf_counter()
    for i in range(degree):
        features = torch.spmm(adj, features)
    precompute_time = perf_counter()-t
    return features, precompute_time

def set_seed(seed, cuda):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if cuda: torch.cuda.manual_seed(seed)


In [35]:
def train_regression(model,
                     train_features, train_labels,
                     val_features, val_labels,
                     epochs, weight_decay, lr):

    optimizer = optim.Adam(model.parameters(), lr,
                           weight_decay=weight_decay)
    t = perf_counter()
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        output = model(train_features)
        loss_train = F.cross_entropy(output, train_labels)
        loss_train.backward()
        optimizer.step()
    train_time = perf_counter()-t

    with torch.no_grad():
        model.eval()
        output = model(val_features)
        acc_val = accuracy(output, val_labels)

    return model, acc_val, train_time

def test_regression(model, test_features, test_labels):
    model.eval()
    return accuracy(model(test_features), test_labels)
def run(dataset="cora", cuda=False, degree=2, weight_decay=0.005, lr=0.2, epochs=100):
  # setting random seeds
  set_seed(42, False)

  adj, features, labels, idx_train, idx_val, idx_test = load_citation(dataset, cuda)
  model = SGC(nfeat=features.size(1),
                    nclass=labels.max().item()+1)

  features, precompute_time = sgc_precompute(features, adj, degree)


  model, acc_val, train_time = train_regression(model, features[idx_train], labels[idx_train], features[idx_val], labels[idx_val],
                    epochs, weight_decay, lr)
  acc_test = test_regression(model, features[idx_test], labels[idx_test])
  print("Hyperparams: degree:{}, weight_decay:{}, lr:{}, epochs:{}".format(degree, weight_decay, lr, epochs))
  print("Validation Accuracy: {:.4f} Test Accuracy: {:.4f}".format(acc_val, acc_test))
  print("Pre-compute time: {:.4f}s, train time: {:.4f}s, total: {:.4f}s\n".format(precompute_time, train_time, precompute_time+train_time))


In [ ]:
import numpy as np

In [36]:
for i in range(1, 10):
  run(degree=i)
# for i in np.arange(0.005, 0.1, 0.005):
#   run(weight_decay=i)

Hyperparams: degree:1, weight_decay:0.005, lr:0.2, epochs:100
Validation Accuracy: 0.7060 Test Accuracy: 0.7380
Pre-compute time: 0.0105s, train time: 0.0833s, total: 0.0938s
Hyperparams: degree:2, weight_decay:0.005, lr:0.2, epochs:100
Validation Accuracy: 0.7620 Test Accuracy: 0.7740
Pre-compute time: 0.0227s, train time: 0.0894s, total: 0.1121s
Hyperparams: degree:3, weight_decay:0.005, lr:0.2, epochs:100
Validation Accuracy: 0.7660 Test Accuracy: 0.7670
Pre-compute time: 0.0360s, train time: 0.0777s, total: 0.1137s
Hyperparams: degree:4, weight_decay:0.005, lr:0.2, epochs:100
Validation Accuracy: 0.7620 Test Accuracy: 0.7700
Pre-compute time: 0.0508s, train time: 0.0814s, total: 0.1322s
Hyperparams: degree:5, weight_decay:0.005, lr:0.2, epochs:100
Validation Accuracy: 0.7620 Test Accuracy: 0.7710
Pre-compute time: 0.0591s, train time: 0.0760s, total: 0.1351s
Hyperparams: degree:6, weight_decay:0.005, lr:0.2, epochs:100
Validation Accuracy: 0.7580 Test Accuracy: 0.7730
Pre-compute t

In [ ]:
def hyper_tuning_plot(x, parm):
  # exec("%s = %d" % (parm, 1))
  res1, res2, res3 = [], [], []
  for i in x:
    if parm == "degree":
      acc_train, acc_val, acc_test = run(degree=i)
    if parm == "weight_decay":
      acc_train, acc_val, acc_test = run(weight_decay=i)
    if parm == "epochs":
      acc_train, acc_val, acc_test = run(epochs=i)
    if parm == "lr":
      acc_train, acc_val, acc_test = run(lr=i)
    if parm == "dropout":
      acc_train, acc_val, acc_test = run(dropout=i)
    if parm == "hidden":
      acc_train, acc_val, acc_test = run(hidden=i)    

    res1.append(acc_val)
    res2.append(acc_test)
    res3.append(acc_train)
  
  plt.plot(x, res1, label="val")
  plt.plot(x, res2, label="test")
  plt.plot(x, res3, label="train")
  plt.xlabel(parm)
  plt.ylabel("accuracy")
  # plt.title(parm)
  plt.legend()
  plt.show()